In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('UUCF Assignment Spreadsheet.xls', sheet_name='movie-row', index_col=0)
#df_users = pd.read_excel('UUCF Assignment Spreadsheet.xls', sheet_name='user-row', index_col=0)
#df_corr = pd.read_excel('UUCF Assignment Spreadsheet.xls', sheet_name='correlations')

In [3]:
df

,1648,5136,918,2824,3867,860,3712,2968,3525,4323,...,3556,5261,2492,5062,2486,4942,2267,4809,3853,2288
11: Star Wars: Episode IV - A New Hope (1977),NaN,4.5,5.0,4.5,4.0,4.0,NaN,5.0,4.0,5.0,...,4.0,NaN,4.5,4.0,3.5,NaN,NaN,NaN,NaN,NaN
12: Finding Nemo (2003),NaN,5.0,5.0,NaN,4.0,4.0,4.5,4.5,4.0,5.0,...,4.0,NaN,3.5,4.0,2.0,3.5,NaN,NaN,NaN,3.5
13: Forrest Gump (1994),NaN,5.0,4.5,5.0,4.5,4.5,NaN,5.0,4.5,5.0,...,4.0,5.0,3.5,4.5,4.5,4.0,3.5,4.5,3.5,3.5
14: American Beauty (1999),NaN,4.0,NaN,NaN,NaN,NaN,4.5,2.0,3.5,5.0,...,4.0,NaN,3.5,4.5,3.5,4.0,NaN,3.5,NaN,NaN
22: Pirates of the Caribbean: The Curse of the Black Pearl (2003),4.0,5.0,3.0,4.5,4.0,2.5,NaN,5.0,3.0,4.0,...,3.0,1.5,4.0,4.0,2.5,3.5,NaN,5.0,NaN,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9806: The Incredibles (2004),3.5,5.0,3.5,NaN,3.0,3.5,4.5,5.0,3.5,4.0,...,3.5,0.5,5.0,3.5,3.0,NaN,NaN,4.5,NaN,3.5
10020: Beauty and the Beast (1991),3.0,5.0,NaN,NaN,4.0,3.5,NaN,2.0,4.0,3.0,...,4.0,NaN,4.0,2.0,2.5,NaN,NaN,4.0,NaN,NaN
36657: X-Men (2000),NaN,4.5,NaN,NaN,4.0,NaN,4.5,5.0,4.0,5.0,...,3.5,NaN,3.0,4.0,3.5,NaN,NaN,NaN,3.0,4.0
36658: X2: X-Men United (2003),3.5,4.0,NaN,NaN,3.5,NaN,4.5,4.5,4.0,5.0,...,3.5,NaN,2.0,NaN,3.0,4.0,NaN,3.5,NaN,4.0


Identify the top 5 neighbors (the users with the 5 largest, positive correlations) for users 3867 and 89. 

For example, if the target user were #3712, the closest neighbors are 2824 (corr: 0.46291), 3867 (corr: 0.400275), 5062 (corr: 0.247693), 442 (corr: 0.22713), and 3853 (corr: 0.19366). 

Don't forget to exclude the target user (corr: 1.0) from your possible selections.

In [3]:
def get_neighbors(user_id, top_n = 5):
    corr =  df.corr().loc[user_id].sort_values(ascending=False).iloc[1:top_n+1].rename('correlation')
    corr.index.name = 'user_id'
    return corr

In [4]:
get_neighbors(3712)

user_id
2824    0.462910
3867    0.400275
5062    0.247693
442     0.227130
3853    0.193660
Name: correlation, dtype: float64

Create a new worksheet in your spreadsheet, and use it to compute the predictions for each movie for users 3867 and 89 by taking the correlation-weighted average of the ratings of the top-five neighbors (for each target user) for each movie. 

The formal formula for correlation-weighted average is $\displaystyle \frac{\sum_{n=1}^{5} r_n  w_n}{\sum_{n=1}^{5} w_n}$ . 

Remember, you will need to make sure that your weight for each contributed rating is the user-user correlation when that neighbor has rated the movie, but 0 when the neighbor has not rated the movie).

In [5]:
def get_predictions(user_id, top_n = 5):
    neighbours = get_neighbors(user_id, top_n)
    weighted_sum = df[neighbours.index].mul(neighbours) 
    denominator = df[neighbours.index].notnull().mul(neighbours).sum(axis=1)
    predictions = weighted_sum.sum(axis=1) / denominator
    return predictions.sort_values(ascending=False)


In [6]:
get_predictions(3867).head()

1891: Star Wars: Episode V - The Empire Strikes Back (1980)    4.760291
155: The Dark Knight (2008)                                    4.551454
122: The Lord of the Rings: The Return of the King (2003)      4.507637
77: Memento (2000)                                             4.472487
121: The Lord of the Rings: The Two Towers (2002)              4.400194
dtype: float64

In [7]:
get_predictions(89).head()

238: The Godfather (1972)               4.894124
278: The Shawshank Redemption (1994)    4.882194
807: Seven (a.k.a. Se7en) (1995)        4.774093
275: Fargo (1996)                       4.770944
424: Schindler's List (1993)            4.729056
dtype: float64

### Part 2 - Normalization

This time, however, use the normalization formula: $\displaystyle \bar{r}_u + \frac{\sum_{n=1}^{5}  \bigl(r_n - \bar{r}n\bigr),w_n} {\sum_{n=1}^{5} w_n} $

In [ ]:
def get_predictions_normalized(user_id, top_n = 5):
    r_u = df[user_id].mean()
    neighbours = get_neighbors(user_id, top_n)

    r_n = df[neighbours.index].mean()

    weighted_sum_norm = (df[neighbours.index] - r_n).mul(neighbours)
    denominator = df[neighbours.index].notnull().mul(neighbours).sum(axis=1)

    predictions = r_u + (weighted_sum_norm.sum(axis=1) / denominator)
    return predictions.sort_values(ascending=False)



In [9]:
get_predictions_normalized(3867).head()

1891: Star Wars: Episode V - The Empire Strikes Back (1980)    5.245509
155: The Dark Knight (2008)                                    4.856770
77: Memento (2000)                                             4.777803
275: Fargo (1996)                                              4.771538
807: Seven (a.k.a. Se7en) (1995)                               4.655569
dtype: float64

In [10]:
get_predictions_normalized(89).head()

238: The Godfather (1972)               5.322015
278: The Shawshank Redemption (1994)    5.261424
275: Fargo (1996)                       5.241111
807: Seven (a.k.a. Se7en) (1995)        5.201984
424: Schindler's List (1993)            5.199223
dtype: float64